#### DNN기반 회귀 모델 구현
- 데이터셋 : iris.csv
- 피쳐/속성 : 3개 Sepal_length, Sepal_Width, Petal_Length
- 타겟/라벨 : 1개 Petal_Width
- 학습-방법 : 지도학습 > 회귀
- 알고리즘 : 인공신경망(ANN) -> MLP, DNN

[1] 모듈 로딩 및 데이터 준비
<hr>

In [2]:
# 모듈 로딩
# - Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# - 데이터 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [3]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 만들어두기
print(f'torch v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

def checkversion(x):
    version = x + '.__version__'
    print(f'{x} v.{version}')

torch v.2.4.1
pandas v.2.0.3


In [4]:
### 데이터 로딩
DATA_FILE=r'C:\VSCode\KDT\Torch_DL\Data\iris.csv'

### CSV >>> DF
irisDF = pd.read_csv(DATA_FILE,usecols=[0,1,2,3])

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매개 변수   : 층별 입출력 개수 고정하므로 필요 없음!
- 속성 / 필드 : featuresDF, targetDF, n_rows, n_featuers
- 기능 역할   : __init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)

- 클래스 구조
    * 입력층 : 입력 3개(피쳐)   출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개        출력 30개(퍼셉트론/뉴런 30개 존재)
    * 출력층 : 입력 30개        출력 1개(너비값)

- 활성화 함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [5]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3,10)
        self.hd_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self,x):
        #            x: 학습하는 데이터

        # - 입력층
        y=self.in_layer(x)    # f1w1 + f2w2 + f3w3 + b => 입력층이 10개라서 이런 애들이 10개 나옴.
        y=F.relu(y)           # relu => y 값의 범위 : 0 <= y

        # - 은닉층 : 10개의 숫자 값(>=0)
        y = self.hd_layer(y)  # f21w11+f22w12....+f210w210+b,....
        y=F.relu(y)           # relu => y 값의 범위 : 0 <= y

        # - 출력층 : 30개의 숫자 값(>=0) 회귀이므로 바로 반환(return)
        return self.out_layer(y)     #f31w31+ ....+f330W330+b

In [6]:
### 모델 인스턴스 생성
model = IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [7]:
 ## 모댈 사용 메모리 정보 확인
summary(model, input_size=(10,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐_개수 : 3개
- 타겟_개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.dataset
- 속성_필드   : featureDF, targetDF, n_rows, n_features
- 필수메서드 
    * _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * _ _len_ _(self)  : 데이터의 개수 반환
    * _ _getitem_ _(self,index) : 특정 인덱스의 피쳐와 타겟 반환 

In [8]:
class IrisDataset(Dataset):

    def __init__(self,featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        # 밑에는 해도 그만 안해도 그만
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows
    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 변환
        return featureTS, targetTS

In [9]:
## 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]  # 2D (150,3)
targetDF = irisDF[irisDF.columns[-1:]]   # 2D (150,1)

# - 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF,targetDF)

[4] 학습 준비
- 학습 횟수 : EPOCH        <- 처음~끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE   <- 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE       <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학습률(LR): 가중치와 절편 업데이트시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1 (하이퍼파라미터)

In [10]:
## 학습 진행 관련 설정
EPOCH = 1 # 연산이나 타입이 안 맞을수도 있으니 처음에는 작게 돌려보자
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001 # => LR이 작으면 작을수록 촘촘하게 내려감.

- 인스턴스/객체 : 모델, 데이터셋, 최적화, (손실함수 , 성능지표) => 선택사항

In [11]:
# 모델 인스턴스
model=IrisRegModel()
# 데이터셋 인스턴스
X_train,X_test,y_train,y_test = train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val= train_test_split(X_train,y_train,random_state=1)

trainDS = IrisDataset(X_train,y_train)
testDS = IrisDataset(X_test,y_test)
valDS = IrisDataset(X_val,y_val)



# 데이터로더 인스턴스
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)

In [12]:
## [테스트] 데이터로더와 데이터셋 체크
for feature, target in trainDS:
    print(feature.shape,target.shape)
    break

torch.Size([3]) torch.Size([1])


In [13]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(),lr=LR)

# 손실함수 인스턴스 => 회귀 mse, mae, rmse...
regLoss = nn.MSELoss()

[5] 학습 진행

In [14]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요

LOSS_HISTORY, SCORE_HISTORY=[[],[]],[[],[]]


for epoch in range(EPOCH):
    # 배치크기만큼 데이터 로딩해서 학습진행

    loss_total, score_total =0,0
    for featureTS, targetTS in trainDL:
        # 학습 모드로 모델 설정
        model.train()

        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pre_y,targetTS)
        loss_total += loss.item() # Tensor니까 item해줘야 값이 들어감.

        # 성능평가 계산
        score = R2Score()(pre_y,targetTS)
        score_total +=score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모데레 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론 / 평가
        pre_val = model(val_featureTS)

        # 손실
        loss_val=regLoss(pre_val,val_targetTS)

        # 성능평가
        score_val = R2Score()(pre_val,val_targetTS)

    
    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_CNT)
    SCORE_HISTORY[0].append(score_total/BATCH_CNT)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)


##### memo
테스트 & 검정 상태
설정된 W,b
검증 및 테스트용 데이터셋 
예측값 추출

** 사용되지 않는 기능들 OFF
- W,b 업데이트 X
-> 기능 OFF Auto_grade 엔진 --- model.eval()
-> W,b 텐서 requierd_grade=True---